ImportError: No module named folium

In [3]:
!pip install folium

In [39]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
import folium
# import seaborn as sns
from folium.plugins import HeatMap
pd.set_option("display.max_rows",1300)
pd.set_option("display.max_columns",12000)
pd.set_option('display.max_rows',200000)
pd.set_option('display.width', 500)
# df = pd.read_csv('denverairbnb/listings.csv')
# # print(df.describe())
# df = df.filter(items=['availability_30','availability_60','availability_90','availability_365',
#                       'price','cleaning_fee','security_deposit','accomodates','bedrooms',
#                       'bathrooms','property_type','room_type','latitude','longitude','zipcode'])
# pr = df['price'].astype(float)
# # p_small = p[p <= 500]
# zips = df['zipcode'].unique()
# zc_dict = {}
# for i in zips:
#     zip_specific=df[df.zipcode==i]
#     zc_dict.update({i:zip_specific})
# p_small = p[p <= 500]
df = pd.read_csv('../src/dedup_df.csv')
df1 = pd.read_csv('../src/real_res.csv')
df2 = pd.read_csv('../src/prop_cost.csv')

# means = {}
# lens = {}
# for i in zc_dict:
#     p = zc_dict.get(i)
#     p = p['price'].astype(float)
#     means.update({i:np.mean(p)})
#     lens.update({i:len(p)})
# std = np.std(p)
# x = np.linspace(0, np.max(p), 200)
denver_map = folium.Map(location=[39.73782,-104.971338],
                        zoom_start=13,
                        tiles="Cartodbpositron")

#Heat_map
heat_map = folium.FeatureGroup(name = 'heat_map')
max_amount = float(1000)
heat_map.add_child( HeatMap( list(zip(df['latitude'].values, df['longitude'].values)),
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=5.5, blur=3.5,
                   max_zoom=1,
                 ))
denver_map.add_child(heat_map)

In [50]:
from folium import plugins
def map_points(df, lat_col='latitude', lon_col='longitude', zoom_start=11, 
               plot_points=False, pt_radius=15, 
               draw_heatmap=False, heat_map_weights_col=None, 
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay

    Arg:
        df: dataframe containing points to maps
        lat_col: Column containing latitude (string)
        lon_col: Column containing longitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lat = df[lat_col].median()
    middle_lon = df[lon_col].median()

    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=pt_radius,
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lat_col, lon_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lat_col, lon_col, heat_map_weights_col]

        stations = df[cols_to_pull].as_matrix()
        curr_map.add_children(plugins.HeatMap(stations, radius=heat_map_radius))

    return curr_map

In [51]:
map_points(df, draw_heatmap=True, plot_points=False,heat_map_weights_normalize=True,
          heat_map_weights_col='Reservations_In_Next_90_Days')

/Users/co/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/co/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [36]:
map_points(df2, draw_heatmap=True, plot_points=False,heat_map_weights_normalize=True,
          heat_map_weights_col='amount')

/Users/co/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/co/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [31]:
map_points(df, draw_heatmap=True, plot_points=False,heat_map_weights_normalize=True,
          heat_map_weights_col='price')

/Users/co/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/co/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
